# Ship360 Chat API - Authentication & Advanced Features

This notebook demonstrates how to authenticate with the Ship360 Chat API and use its more advanced features such as multi-turn conversations, conversation history, and customizing parameters.

## Authentication

The Ship360 Chat API uses API keys for authentication. In production environments, you would obtain an API key from the Ship360 administration portal.

In [ ]:
import os
import json
import requests
from dotenv import load_dotenv
from pprint import pprint

# Load environment variables
load_dotenv()

# Define the base URL for the API
BASE_URL = "http://localhost:8000"

# For production, you would use
# BASE_URL = "https://api.ship360.com"

# Get API key from environment or set directly (for testing only)
API_KEY = os.environ.get("SHIP360_API_KEY", "your-api-key-here")

## Making Authenticated API Calls

Let's create a function to send authenticated requests to the Ship360 Chat API.

In [ ]:
def send_authenticated_message(message, endpoint="/api/v1/chat", parameters=None):
    """Send an authenticated chat message to the API and return the response."""
    url = f"{BASE_URL}{endpoint}"
    headers = {
        "Content-Type": "application/json",
        "Authorization": f"Bearer {API_KEY}"
    }
    
    payload = {"message": message}
    
    # Add optional parameters if provided
    if parameters:
        payload.update(parameters)
    
    try:
        response = requests.post(url, headers=headers, json=payload)
        response.raise_for_status()
        return response.json()
    except requests.exceptions.RequestException as e:
        print(f"Error making request: {e}")
        return None

## Multi-turn Conversations

The Ship360 Chat API supports multi-turn conversations by maintaining context between queries. Let's see how to implement this.

In [ ]:
class Ship360Conversation:
    """A class to manage multi-turn conversations with the Ship360 API."""
    
    def __init__(self, api_key=None, base_url=None):
        self.api_key = api_key or API_KEY
        self.base_url = base_url or BASE_URL
        self.conversation_id = None
        self.messages = []
    
    def send_message(self, message, parameters=None):
        """Send a message in this conversation."""
        url = f"{self.base_url}/api/v1/chat"
        headers = {
            "Content-Type": "application/json",
            "Authorization": f"Bearer {self.api_key}"
        }
        
        payload = {"message": message}
        
        # Add conversation ID if we have one
        if self.conversation_id:
            payload["conversation_id"] = self.conversation_id
        
        # Add optional parameters if provided
        if parameters:
            payload.update(parameters)
        
        try:
            response = requests.post(url, headers=headers, json=payload)
            response.raise_for_status()
            result = response.json()
            
            # Store conversation ID if returned
            if "conversation_id" in result:
                self.conversation_id = result["conversation_id"]
            
            # Add to message history
            self.messages.append({"role": "user", "content": message})
            self.messages.append({"role": "assistant", "content": result.get("response", "")})
            
            return result
        except requests.exceptions.RequestException as e:
            print(f"Error making request: {e}")
            return None
    
    def get_conversation_history(self):
        """Get the conversation history."""
        return self.messages

## Example: Using Multi-turn Conversation

Let's demonstrate a multi-turn conversation where context is maintained between messages.

In [ ]:
# Create a new conversation
conversation = Ship360Conversation()

# First message
print("Sending first message...")
response1 = conversation.send_message("I want to ship a package to New York")
if response1:
    print(f"Response: {response1.get('response')}\n")

# Second message - should have context from the first
print("Sending follow-up message...")
response2 = conversation.send_message("How much would it cost for a 3-pound package?")
if response2:
    print(f"Response: {response2.get('response')}\n")

# Third message - continuing the conversation
print("Sending final message...")
response3 = conversation.send_message("What about expedited shipping?")
if response3:
    print(f"Response: {response3.get('response')}\n")

# Show conversation history
print("\nConversation History:")
for i, msg in enumerate(conversation.get_conversation_history()):
    print(f"{msg['role'].capitalize()}: {msg['content']}")
    if i < len(conversation.get_conversation_history()) - 1:
        print("-" * 40)

## Customizing Parameters

The Ship360 Chat API allows customization of various parameters to control the behavior of the AI. Let's explore some common parameters.

In [ ]:
# Example of sending a message with custom parameters
custom_parameters = {
    "temperature": 0.7,  # Controls randomness (0.0-1.0)
    "max_tokens": 150,   # Maximum response length
    "model": "gpt-4",   # Specific model to use
}

response = send_authenticated_message(
    "What are the shipping options for fragile items?", 
    parameters=custom_parameters
)

if response:
    print("Response with custom parameters:")
    print(response.get("response"))

## Advanced Feature: Retrieving Extended Shipping Information

The Ship360 API can provide detailed information about shipping options, carriers, and services. Let's explore this capability.

In [ ]:
def get_shipping_options(origin, destination, weight, dimensions=None, value=None):
    """Get available shipping options for a package."""
    url = f"{BASE_URL}/api/v1/shipping/options"
    headers = {
        "Content-Type": "application/json",
        "Authorization": f"Bearer {API_KEY}"
    }
    
    payload = {
        "origin": origin,
        "destination": destination,
        "weight": weight,
    }
    
    if dimensions:
        payload["dimensions"] = dimensions
    
    if value:
        payload["declared_value"] = value
    
    try:
        response = requests.post(url, headers=headers, json=payload)
        response.raise_for_status()
        return response.json()
    except requests.exceptions.RequestException as e:
        print(f"Error making request: {e}")
        return None

## Example: Comparing Shipping Options

Let's use the function to compare shipping options for different package types.

In [ ]:
# Define some test packages
test_packages = [
    {"origin": "Seattle, WA", "destination": "New York, NY", "weight": 2, "dimensions": {"length": 10, "width": 8, "height": 6}},
    {"origin": "Los Angeles, CA", "destination": "Miami, FL", "weight": 10, "dimensions": {"length": 18, "width": 14, "height": 12}, "value": 500},
    {"origin": "Chicago, IL", "destination": "Houston, TX", "weight": 1, "dimensions": {"length": 6, "width": 4, "height": 2}},
]

# Get shipping options for each package
for i, package in enumerate(test_packages):
    print(f"\nPackage {i+1}: {package['weight']}lbs from {package['origin']} to {package['destination']}")
    
    options = get_shipping_options(
        package["origin"],
        package["destination"],
        package["weight"],
        package.get("dimensions"),
        package.get("value")
    )
    
    if options and "shipping_options" in options:
        print(f"Found {len(options['shipping_options'])} shipping options:")
        for option in options["shipping_options"]:
            print(f"- {option['carrier']} {option['service']}: ${option['rate']} ({option['delivery_time']} days)")
    else:
        print("No shipping options found or API error occurred.")

## Conclusion

In this notebook, we've explored the advanced features of the Ship360 Chat API, including:

1. Authentication with API keys
2. Multi-turn conversations with context maintenance
3. Customizing parameters for different use cases
4. Retrieving detailed shipping information and options

These capabilities can be integrated into a wide range of applications, from customer service chatbots to shipping calculators and order tracking systems.